# Get Time Series for the Data Cube

In [1]:
import sys
sys.path.append("helpers")
import helpers.helpers as helpers
%matplotlib inline

def on_button_clicked(b): 
    h = helpers.Helpers(out)
    data = h.data_to_csv(product.value,
                  subproduct.value,
                  latitude.value,
                  longitude.value,
                  start.value,
                  end.value)

wid = helpers.Widgets()
start, end = wid.get_date_widgets()
latitude, longitude = wid.get_lat_lon_widgets()
product = wid.product()
subproduct = wid.subproduct()
button = wid.set_up_button(on_button_clicked)
out = wid.display_output()

# Define a function that updates the content of y based on what we select for x
def update(*args):
    subproduct.options = wid.get_subproduct_list(product.value)

update()
product.observe(update)

wid.display_widget([product, subproduct, latitude, longitude, start, end, button])
out = wid.display_output()

Output()

Dropdown(description='Product:', layout=Layout(height='30px', width='400px'), options=('chirps', 'tamsat', 'er…

Dropdown(description='Subproduct:', layout=Layout(height='30px', width='400px'), options=('rfe',), value='rfe'…

BoundedFloatText(value=7.5, description='Latitude (y):', layout=Layout(height='30px', width='400px'), max=90.0…

BoundedFloatText(value=7.5, description='Longitude (x):', layout=Layout(height='30px', width='400px'), max=180…

DatePicker(value=datetime.datetime(2000, 1, 1, 0, 0), description='Start Date: ', layout=Layout(height='30px',…

DatePicker(value=datetime.datetime(2000, 2, 1, 0, 0), description='EndDate: ', layout=Layout(height='30px', wi…

LoadedButton(button_style='primary', description='Get Data', layout=Layout(height='30px', width='400px'), styl…

Output()